In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchsummary
from torch.utils.data import Dataset, DataLoader
from skimage.io import imread
from skimage.transform import resize
import torchvision.transforms as transforms
import os
import numpy as np
import pandas as pd
import tensorly as tl

In [2]:
class resnet_test_data(Dataset):
    def __init__(self, dir):
        self.files = sorted(os.listdir(dir))[:200]
        self.dir = dir
        self.transform = transforms.Compose([transforms.ToTensor()])
        df = pd.read_csv('ILSVRC2010_test_ground_truth.txt', index_col=False, header=None)
        values = np.squeeze(df.values)[:200]
        self.vals = values
    def __getitem__(self, idx):
        img = imread(os.path.join(self.dir, self.files[idx])) 
        class_id = self.vals[idx]
        img = resize(img, (224,224))
        img = self.transform(img)
        
        return torch.tensor(img, dtype=torch.float32), torch.tensor(class_id)
    def __len__(self):
        return len(self.files)

In [3]:
class resnet_train_data(Dataset):
    def __init__(self, dir):
        self.files = sorted(os.listdir(dir))[200:]
        self.dir = dir
        self.transform = transforms.Compose([transforms.ToTensor()])
        df = pd.read_csv('ILSVRC2010_test_ground_truth.txt', index_col=False, header=None)
        values = np.squeeze(df.values)[200:200+len(self.files)]
        self.vals = values
    def __getitem__(self, idx):
        img = imread(os.path.join(self.dir, self.files[idx])) 
        class_id = self.vals[idx]
        img = resize(img, (224,224))
        img = self.transform(img)
        
        return torch.tensor(img, dtype=torch.float32), torch.tensor(class_id)
    def __len__(self):
        return len(self.files)

In [4]:
dset = resnet_test_data("test_slice")
dset_train = resnet_train_data("test_slice/")

In [5]:
len(dset_train)

400

In [6]:
model = torchvision.models.resnet18(pretrained=True)
base_model = torchvision.models.resnet18(pretrained=True)
base_model.cuda()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
data_loader = DataLoader(dset, batch_size=200)
train_loader = DataLoader(dset_train, batch_size=64)

In [8]:
with torch.no_grad():
    for x,y in data_loader:
        x = x.cuda()

        class_labels = np.argmax(base_model(x).cpu().detach().numpy(), axis=1)
        print(class_labels)
        #print(y.numpy())

/home/egor/egor_torch/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


[489 671 823 299 555 987  56 281 738 938 942 299  82 738 457 858 299 666
 462 988 493 858 657 645 133 374 469  71 449  45 467 670 414 644 788 674
 118 527  94  70 621 986 286 112 897 703 457 649 589 645 631 450 786 387
  40 180 674 637 424 111 873 211 777 681 338 417  32 289 841 808 309 788
 892 831 909 422 551 800 393 408 671 776 162 738 467 564  72 187 420 515
 154 652 841 852 439 663 510 975  90 856 584 601 815 936 275 518 652 997
 233 988 939 559 425 570 122 886 116 300 118 894 407 629 755  54 313  11
 800 640 540 564 738 326 322 386 955 793 647 732 582 990 875 804 557 551
 759 309 767 687 447 723 877 451 583 987 772 618 496 324 422 984 375 301
  38  81 757 151 905 973 644 599 418  35  84 233 437 453 311 792 537  25
 488 607 564 286 818 946 669 317 613 751  66  40 961 446 371  71 400  47
 971 546]


In [9]:
true_pred = class_labels

In [10]:
def evaluate_model(restored_conv, true_pred):
    model = torchvision.models.resnet18(pretrained=True)
    model.layer4[1].conv1.weight.data = torch.tensor(restored_conv.reshape(512,512,3,3), dtype=torch.float32)
    _ = model.cuda()
    data_loader = DataLoader(dset, batch_size=200)
    with torch.no_grad():
        for x,y in data_loader:
            x = x.cuda()
            class_labels = np.argmax(model(x).cpu().detach().numpy(), axis=1)
            acc = (class_labels == true_pred).sum()/200
    return acc

In [11]:
def fine_tune():
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    for epoch in range(10):
        for x,y in data_loader:
            with torch.no_grad():
                x = x.cuda()

                class_labels = np.argmax(base_model(x).cpu().detach().numpy(), axis=1)

            torch.cuda.empty_cache()
            optimizer.zero_grad()
            y = model(x)

            loss = criterion(y, torch.tensor(class_labels).cuda())
            loss.backward()
            #print(loss)
            optimizer.step()
    print(loss)

In [12]:
def replace_layers(core_factors, factors):
    layers = [nn.Conv2d(in_channels=512,
                       out_channels=r1,
                       kernel_size=1),
             nn.Conv2d(in_channels=r1,
                      out_channels=r,
                      kernel_size=1),
             nn.Conv2d(in_channels=r,
                      out_channels=r,
                      groups=r,
                      kernel_size=3),
             nn.Conv2d(in_channels=r,
                      out_channels=r2,
                      kernel_size=1),
             nn.Conv2d(in_channels=r2,
                      out_channels=512,
                      kernel_size=1)]
    kernels = [torch.tensor(factors[0].T.reshape(r1, 512,1,1), dtype=torch.float32),
                torch.tensor(core_factors[1].T.reshape(r,r1,1,1), dtype=torch.float32),
                torch.tensor(core_factors[0].T.reshape(r,1, 3,3), dtype=torch.float32),
                torch.tensor(core_factors[2].reshape(r2,r,1,1), dtype=torch.float32),
                torch.tensor(factors[1].reshape(512,r2,1,1), dtype=torch.float32)]
    for i in range(len(kernels)):
        layers[i].weight = nn.Parameter(kernels[i])
    model.layer4.conv1 =  nn.Sequential(*layers)
    model.cuda()

In [13]:
def eval():
    data_loader = DataLoader(dset, batch_size=200)
    for x,y in data_loader:
        with torch.no_grad():
            x = x.cuda()

            class_labels = np.argmax(model(x).cpu().detach().numpy(), axis=1)

    return((class_labels == true_pred).sum()/200.)

In [14]:
acc = {}

In [15]:
for r1 in (30,60,90):
    acc[r1] = {}
    for r2 in (30,60,90):
        acc[r1][r2] = {}
        for R in [100, 120, 150, 180, 240, 250]:
            
            core = np.load(f"./decompositions/core_{r1}_{r2}_{R}.npy")
            factors = []
            for i in range(2):
                factors.append(np.load(f"./decompositions/factor_{i}_{r1}_{r2}_{R}.npy"))

            core_weights = np.load(f"./decompositions/core_weights_{r1}_{r2}_{R}.npy")
            core_factors = []
            for i in range(3):
                core_factors.append(np.load(f"./decompositions/core_factor_{i}_{r1}_{r2}_{R}.npy"))
            core_factors = tuple(core_factors)
            
            model = torchvision.models.resnet18(pretrained=True)
            r = R
            replace_layers(core_factors, factors)
            fine_tune()
            acc[r1][r2][R] = eval()

/home/egor/egor_torch/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  from ipykernel import kernelapp as app


tensor(0.0381, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0629, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0865, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0563, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(nan, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0940, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0624, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0506, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1392, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1365, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0991, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0593, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1066, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.1809, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0900, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(0.0854, device='cuda:0', grad_fn=<NllLossBackward>)
ten

In [18]:
import pickle
with open("acc.pickle", "wb") as f:
    pickle.dump(acc, f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
data_loader = DataLoader(dset, batch_size=64)

In [ ]:
replace_layers(core_factors, factors)

In [19]:
acc

{30: {30: {100: 1.0, 120: 1.0, 150: 1.0, 180: 1.0, 240: 0.0, 250: 0.0},
  60: {100: 1.0, 120: 0.995, 150: 0.995, 180: 0.995, 240: 1.0, 250: 0.995},
  90: {100: 1.0, 120: 0.995, 150: 0.985, 180: 1.0, 240: 0.985, 250: 0.99}},
 60: {30: {100: 0.96, 120: 0.935, 150: 0.3, 180: 0.0, 240: 0.0, 250: 0.0},
  60: {100: 0.985, 120: 1.0, 150: 0.45, 180: 0.99, 240: 1.0, 250: 1.0},
  90: {100: 1.0, 120: 1.0, 150: 0.94, 180: 0.985, 240: 0.93, 250: 1.0}},
 90: {30: {100: 0.015, 120: 0.02, 150: 0.01, 180: 0.06, 240: 0.0, 250: 0.0},
  60: {100: 0.98, 120: 0.975, 150: 0.64, 180: 0.3, 240: 0.87, 250: 0.78},
  90: {100: 0.915, 120: 0.51, 150: 0.745, 180: 0.755, 240: 0.63, 250: 0.79}}}